In [99]:
import getpass
import os
from langchain_community.utilities import SQLDatabase
from dotenv import load_dotenv


load_dotenv('.env')
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")


# Database connection details
DB_HOST = os.getenv("DB_HOST", "localhost")
DB_PORT = os.getenv("DB_PORT", "5432")
DB_NAME = os.getenv("DB_NAME", "your_database_name")
DB_USER = os.getenv("DB_USER", "your_username")
DB_PASSWORD = os.getenv("DB_PASSWORD", "your_password")

engine = f'postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}'

db = SQLDatabase.from_uri(engine)
print(db.dialect)
print(db.get_usable_table_names())
print(db.run("SELECT SUM(PURCHASE_AMT_ACTUAL)/SUM(purchase_amt_original) AS AVERAGE_DISCOUNT_RATE FROM PURCHASE_DATA A WHERE EXTRACT(MONTH FROM A.DATE::DATE) = 7 AND EXTRACT(YEAR FROM A.DATE::DATE) = 2024 AND product_id IN (SELECT product_id FROM CLICK_DATA WHERE brand_name = 'Nike');"))

postgresql
['click_data', 'purchase_data']
[(0.9567846607669617,)]


In [3]:
a = db.run("SELECT * FROM PURCHASE_DATA LIMIT 10;")

In [9]:
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "How much was purchased on 2024-06-30"})
response

"SELECT SUM(purchase_amt_actual) AS total_purchase_amount\nFROM purchase_data\nWHERE date = '6/30/2024'"

In [10]:
db.run(response)

'[(734.4,)]'

In [11]:
chain.get_prompts()[0].pretty_print()

You are a PostgreSQL expert. Given an input question, first create a syntactically correct PostgreSQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per PostgreSQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURRENT_DATE function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLR

In [12]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

execute_query = QuerySQLDataBaseTool(db=db)  # does the db.run part for you
write_query = create_sql_query_chain(llm, db)
chain = write_query | execute_query
chain.invoke({"question": "How much was purchased on 2024-06-30"})

/tmp/ipykernel_1810/2892724753.py:3: LangChainDeprecationWarning: The class `QuerySQLDataBaseTool` was deprecated in LangChain 0.3.12 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-community package and should be used instead. To use it run `pip install -U :class:`~langchain-community` and import as `from :class:`~langchain_community.tools import QuerySQLDatabaseTool``.
  execute_query = QuerySQLDataBaseTool(db=db)


'[(734.4,)]'

In [15]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
execute_query = QuerySQLDataBaseTool(db=db)  # does the db.run part for you
write_query = create_sql_query_chain(llm, db)

answer = answer_prompt | llm | StrOutputParser()
chain = (
    RunnablePassthrough.assign(query=write_query).assign(result=itemgetter("query") | execute_query) | answer
)

chain.invoke({"question": "How much was purchased on 2024-06-30"})

'The total purchase amount on June 30, 2024 was $734.40.'

In [21]:
RunnablePassthrough.assign(query=write_query).assign(result=itemgetter("query") | execute_query) | answer

RunnableAssign(mapper={
  query: RunnableAssign(mapper={
           input: RunnableLambda(...),
           table_info: RunnableLambda(...)
         })
         | RunnableLambda(lambda x: {k: v for k, v in x.items() if k not in ('question', 'table_names_to_use')})
         | PromptTemplate(input_variables=['input', 'table_info'], input_types={}, partial_variables={'top_k': '5'}, template='You are a PostgreSQL expert. Given an input question, first create a syntactically correct PostgreSQL query to run, then look at the results of the query and return the answer to the input question.\nUnless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per PostgreSQL. You can order the results to return the most informative data in the database.\nNever query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote the

# Agents

In [ ]:
import getpass
import os
from langchain_community.utilities import SQLDatabase
from dotenv import load_dotenv
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI



load_dotenv('.env')
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")


# Database connection details
DB_HOST = os.getenv("DB_HOST", "localhost")
DB_PORT = os.getenv("DB_PORT", "5432")
DB_NAME = os.getenv("DB_NAME", "your_database_name")
DB_USER = os.getenv("DB_USER", "your_username")
DB_PASSWORD = os.getenv("DB_PASSWORD", "your_password")

engine = f'postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}'

db = SQLDatabase.from_uri(engine)
print(db.dialect)
print(db.get_usable_table_names())
print(db.run("SELECT * FROM PURCHASE_DATA LIMIT 10;"))

In [22]:
from langchain_community.agent_toolkits import create_sql_agent
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [29]:
agent_executor.invoke(
    {
        "input": "Top 10 products with the highest purchase_amt_actual?"
    }
)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


click_data, purchase_data
Invoking: `sql_db_schema` with `{'table_names': 'purchase_data'}`



CREATE TABLE purchase_data (
	purchase_id BIGINT, 
	date TEXT, 
	user_id BIGINT, 
	product_id BIGINT, 
	purchase_qty BIGINT, 
	purchase_amt_original BIGINT, 
	purchase_amt_actual DOUBLE PRECISION, 
	product_actual_price DOUBLE PRECISION, 
	product_original_price BIGINT
)

/*
3 rows from purchase_data table:
purchase_id	date	user_id	product_id	purchase_qty	purchase_amt_original	purchase_amt_actual	product_actual_price	product_original_price
1	6/30/2024	78	2	2	918	734.4	367.2	459
2	5/3/2024	98	3	2	198	178.2	89.1	99
3	5/10/2024	63	2	3	1377	1101.6	367.2	459
*/
Invoking: `sql_db_query` with `{'query': 'SELECT product_id, purchase_amt_actual FROM purchase_data ORDER BY purchase_amt_actual DESC LIMIT 10'}`


[(10, 2411.5), (5, 2020.5), (5, 2020.5), (5, 2020.5), (10, 1929.2), (10, 1929.2), (10, 1929.2), (5, 1616.

{'input': 'Top 10 products with the highest purchase_amt_actual?',
 'output': 'The top 10 products with the highest purchase_amt_actual are as follows:\n\n1. Product ID: 10, Purchase Amount Actual: 2411.5\n2. Product ID: 5, Purchase Amount Actual: 2020.5\n3. Product ID: 5, Purchase Amount Actual: 2020.5\n4. Product ID: 5, Purchase Amount Actual: 2020.5\n5. Product ID: 10, Purchase Amount Actual: 1929.2\n6. Product ID: 10, Purchase Amount Actual: 1929.2\n7. Product ID: 10, Purchase Amount Actual: 1929.2\n8. Product ID: 5, Purchase Amount Actual: 1616.4\n9. Product ID: 5, Purchase Amount Actual: 1616.4\n10. Product ID: 5, Purchase Amount Actual: 1616.4'}

In [100]:
examples = [
    {
        "input": "Which brand was sold the most?",
        "query": "SELECT B.BRAND_NAME FROM PURCHASE_DATA A LEFT JOIN CLICK_DATA B ON A.PRODUCT_ID = B.PRODUCT_ID GROUP BY B.BRAND_NAME ORDER BY SUM(A.PURCHASE_AMT_ACTUAL) DESC LIMIT 1;",
    },
    {
        "input": "Which category was sold the most?",
        "query": "SELECT B.PRODUCT_CATEGORY FROM PURCHASE_DATA A LEFT JOIN CLICK_DATA B ON A.PRODUCT_ID = B.PRODUCT_ID GROUP BY B.PRODUCT_CATEGORY ORDER BY SUM(A.PURCHASE_AMT_ACTUAL) DESC LIMIT 1;",
    },    
    {
        "input": "What is the average discount rate made by nike?",
        "query": "SELECT B.BRAND_NAME, SUM(A.PURCHASE_AMT_ACTUAL)/SUM(A.purchase_amt_original) AS AVERAGE_DISCOUNT_RATE FROM PURCHASE_DATA A LEFT JOIN CLICK_DATA B ON A.PRODUCT_ID = B.PRODUCT_ID WHERE B.BRAND_NAME = 'Nike' GROUP BY B.BRAND_NAME;",
    },  
    {
        "input": "What is the average discount rate made by addidas?",
        "query": "SELECT B.BRAND_NAME, SUM(A.PURCHASE_AMT_ACTUAL)/SUM(A.purchase_amt_original) AS AVERAGE_DISCOUNT_RATE FROM PURCHASE_DATA A LEFT JOIN CLICK_DATA B ON A.PRODUCT_ID = B.PRODUCT_ID WHERE B.BRAND_NAME = 'Addidas' GROUP BY B.BRAND_NAME;",
    },  
    {
        "input": "What is the average discount rate made by nike during july 2024?",
        "query": "SELECT SUM(PURCHASE_AMT_ACTUAL)/SUM(purchase_amt_original) AS AVERAGE_DISCOUNT_RATE FROM PURCHASE_DATA A WHERE EXTRACT(MONTH FROM A.DATE::DATE) = 7 AND EXTRACT(YEAR FROM A.DATE::DATE) = 2024 AND product_id IN (SELECT product_id FROM CLICK_DATA WHERE brand_name = 'Nike');",
    },    
]

In [101]:
from langchain_community.vectorstores import FAISS
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(),
    FAISS,
    k=5,
    input_keys=["input"],
)

In [102]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    FewShotPromptTemplate,
    MessagesPlaceholder,
    PromptTemplate,
    SystemMessagePromptTemplate,
)

system_prefix = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the given tools. Only use the information returned by the tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

If the question does not seem related to the database, just return "I don't know" as the answer.

Here are some examples of user inputs and their corresponding SQL queries:"""

few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=PromptTemplate.from_template(
        "User input: {input}\nSQL query: {query}"
    ),
    input_variables=["input", "dialect", "top_k"],
    prefix=system_prefix,
    suffix="",
)

In [103]:
full_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate(prompt=few_shot_prompt),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

In [104]:
# Example formatted prompt
prompt_val = full_prompt.invoke(
    {
        "input": "Which brand was sold the most?",
        "top_k": 5,
        "dialect": "postgres",
        "agent_scratchpad": [],
    }
)
print(prompt_val.to_string())

System: You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct postgres query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the given tools. Only use the information returned by the tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

If the question does not seem related to the database, just return "I don'

In [105]:
agent = create_sql_agent(
    llm=llm,
    db=db,
    prompt=full_prompt,
    verbose=True,
    agent_type="openai-tools",
)

In [106]:
agent.invoke({"input": "What is the average discount rate made by nike during july 2024?"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_query` with `{'query': "SELECT SUM(PURCHASE_AMT_ACTUAL)/SUM(purchase_amt_original) AS AVERAGE_DISCOUNT_RATE FROM PURCHASE_DATA A WHERE EXTRACT(MONTH FROM A.DATE::DATE) = 7 AND EXTRACT(YEAR FROM A.DATE::DATE) = 2024 AND product_id IN (SELECT product_id FROM CLICK_DATA WHERE brand_name = 'Nike');"}`


[(0.9567846607669617,)]The average discount rate made by Nike during July 2024 is approximately 95.68%.

> Finished chain.


{'input': 'What is the average discount rate made by nike during july 2024?',
 'output': 'The average discount rate made by Nike during July 2024 is approximately 95.68%.'}